In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModel,AutoTokenizer,AutoConfig
from transformers import AdamW, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
import datasets
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import log_loss
from pathlib import Path
import torch.nn.functional as F
import os
import gc
from joblib import Parallel, delayed
import joblib
import torch.nn as nn
import warnings
import time
from sklearn.metrics import log_loss
from tqdm.auto import tqdm
from torch.cuda import amp
from functools import partial
from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs
import random
import warnings
warnings.filterwarnings("ignore")
%env TOKENIZERS_PARALLELISM=false

os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

env: TOKENIZERS_PARALLELISM=false


In [2]:
import transformers
transformers.logging.set_verbosity_error()

In [3]:
def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

In [4]:
class CFG:
    batch_size=16
    epochs=2
    num_workers=2
    seed = 43
    hidden_dropout_prob = 0
    fp16=True
    target_columns = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    gradient_checkpointing_enable = False
    inference = True
    reinit_weights = False
    n_tokens = 40
    tokenizer_deberta_v3_large = AutoTokenizer.from_pretrained("../input/fb3models/microsoft-deberta-v3-large/tokenizer")
    tokenzier_deberta_v3_base = AutoTokenizer.from_pretrained("../input/fb3models/microsoft-deberta-v3-base/tokenizer")
    tokenizer_deberta_v2_xlarge = AutoTokenizer.from_pretrained("../input/fb3models/microsoft-deberta-v2-xlarge/tokenizer")
    tokenizer_deberta_v2_xxlarge = AutoTokenizer.from_pretrained("../input/fb3models/microsoft-deberta-v2-xxlarge/tokenizer")
    tokenizer_deberta_large = AutoTokenizer.from_pretrained("../input/deberta-large")
    tokenizer_electra_large = AutoTokenizer.from_pretrained("../input/google-electra-large-discriminator")

In [5]:
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

In [6]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

In [7]:
#Getting the prompt tuning soft embeddings
class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to 
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens, 
                                                                               random_range, 
                                                                               initialize_from_vocab))
            
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [8]:
class Model(nn.Module):
    def __init__(self, model_name):
        super(Model, self).__init__()
        
        self.model_name = model_name

        hidden_dropout_prob: float = CFG.hidden_dropout_prob
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(model_name)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "attention_probs_dropout_prob" : hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
                "num_labels": 6,
            }
        )
        
        self.config = config
        
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        if CFG.gradient_checkpointing_enable:
            self.transformer.gradient_checkpointing_enable()
        
        self.output = nn.Linear(config.hidden_size, 6)
        self.loss = nn.SmoothL1Loss(reduction = "mean")

        if CFG.reinit_weights:
            self.init_weights_(CFG.reinit_layers)

    def init_weights_(self, reinit_layers):
        for layer in self.transformer.encoder.layer[-reinit_layers:]:
            for module in layer.modules():
                if isinstance(module, nn.Linear):
                    module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                    if module.bias is not None:
                        module.bias.data.zero_()
                elif isinstance(module, nn.Embedding):
                    module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                    if module.padding_idx is not None:
                        module.weight.data[module.padding_idx].zero_()
                elif isinstance(module, nn.LayerNorm):
                    module.bias.data.zero_()
                    module.weight.data.fill_(1.0)

    def get_grouped_llrd_optimizer_scheduler(self, num_train_steps):
        no_decay = ["bias", "LayerNorm.weight"]
      # initialize lr for task specific layer
        optimizer_grouped_parameters = [
              {
                  "params": [p for n, p in self.named_parameters() if "output.weight" in n or "output.bias" in n],
                  "weight_decay": 0.0,
                  "lr": CFG.lr,
              },
          ]
          # initialize lrs for every layer
        num_layers = self.config.num_hidden_layers
        layers = [self.transformer.embeddings] + list(self.transformer.encoder.layer)
        layers.reverse()
        lr = CFG.lr
        for layer in layers:
            lr *= CFG.llrd
            optimizer_grouped_parameters += [
                  {
                      "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                      "weight_decay": 0.001,
                      "lr": lr,
                  },
                  {
                      "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                      "weight_decay": 0.0,
                      "lr": lr,
                  },
              ]
            opt = torch.optim.AdamW(optimizer_grouped_parameters, lr=CFG.lr)
            sch = get_polynomial_decay_schedule_with_warmup(
              opt,
              num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
              num_training_steps=num_train_steps,
              last_epoch=-1,
              )
        return opt, sch
        
    def get_optimizer_scheduler(self, num_train_steps):
        param_optimizer = list(self.named_parameters())
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        opt = torch.optim.AdamW(optimizer_parameters, lr=CFG.lr)
        sch = get_polynomial_decay_schedule_with_warmup(
            opt,
            num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
            num_training_steps=num_train_steps,
            last_epoch=-1,
        )
        return opt, sch

    def forward(self, ids, mask, token_type_ids=None, targets=None):
        
        if token_type_ids is not None:
            transformer_out = self.transformer( ids, mask, token_type_ids )
        else:
            transformer_out = self.transformer(ids, mask)
        sequence_output = transformer_out.last_hidden_state[:,0,:]
        
        logits = self.output(sequence_output)
        if targets is not None:
            loss = self.loss(logits, targets)
        else:
            loss = None

        return loss, logits

In [9]:
class PromptModel(nn.Module):
    def __init__(self, model_name):
        super(PromptModel, self).__init__()
        
        self.model_name = model_name

        hidden_dropout_prob: float = CFG.hidden_dropout_prob
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(model_name)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "attention_probs_dropout_prob" : hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
                "num_labels": 6,
            }
        )
        
        self.config = config
        
        #Using Prompt Tuning
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        s_wte = SoftEmbedding(self.transformer.get_input_embeddings(), 
                      n_tokens=CFG.n_tokens, 
                      initialize_from_vocab=True)
        self.transformer.set_input_embeddings(s_wte)
        
        if CFG.gradient_checkpointing_enable:
            self.transformer.gradient_checkpointing_enable()
        
        self.layer_nums = list(range(self.config.num_hidden_layers - 6, self.config.num_hidden_layers))
            
        self.freeze()
        
        self.output = nn.Linear(config.hidden_size, 6)
        # self.meanpooling = MeanPooling()
        self.loss = nn.SmoothL1Loss()

        if CFG.reinit_weights:
            self.init_weights_(CFG.reinit_layers)

    def init_weights_(self, reinit_layers):
        for layer in self.transformer.encoder.layer[-reinit_layers:]:
            for module in layer.modules():
                if isinstance(module, nn.Linear):
                    module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                    if module.bias is not None:
                        module.bias.data.zero_()
                elif isinstance(module, nn.Embedding):
                    module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
                    if module.padding_idx is not None:
                        module.weight.data[module.padding_idx].zero_()
                elif isinstance(module, nn.LayerNorm):
                    module.bias.data.zero_()
                    module.weight.data.fill_(1.0)

    def get_grouped_llrd_optimizer_scheduler(self, num_train_steps):
        no_decay = ["bias", "LayerNorm.weight"]
        top_params = list(self.output.named_parameters())
      # initialize lr for task specific layer
        optimizer_grouped_parameters = [
              {
                  "params": [p for n, p in top_params if not any(nd in n for nd in no_decay)],
                  "weight_decay": 0.001,
                  "lr": CFG.lr,
              },
              {
                  "params": [p for n, p in top_params if any(nd in n for nd in no_decay)],
                  "weight_decay": 0.0,
                  "lr": CFG.lr,
              },
          ]
      # initialize lrs for every layer
        num_layers = self.config.num_hidden_layers
        layers = [self.transformer.embeddings] + list(self.transformer.encoder.layer)
        layers.reverse()
        lr = CFG.lr
        for layer in layers:
            lr *= CFG.llrd
            optimizer_grouped_parameters += [
              {
                  "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                  "weight_decay": 0.001,
                  "lr": lr,
              },
              {
                  "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                  "weight_decay": 0.0,
                  "lr": lr,
              },
          ]
        opt = torch.optim.AdamW(optimizer_grouped_parameters, lr=CFG.lr)
        sch = get_polynomial_decay_schedule_with_warmup(
          opt,
          num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
          num_training_steps=num_train_steps,
          last_epoch=-1,
          )
        return opt, sch
    
    
    def freeze(self):
        for n,param in self.transformer.named_parameters():
            if "learned_embedding" not in n and not any([str(ln) in n for ln in self.layer_nums]) and not any([str(ln) in n for ln in["transformer.encoder.rel_embeddings.weight","transformer.encoder.LayerNorm.weight","transformer.encoder.LayerNorm.bias"]]):
                param.requires_grad = False
        
    
    def get_optimizer_scheduler(self, num_train_steps):
        param_optimizer = [('learned_embedding', self.transformer.embeddings.word_embeddings.learned_embedding)] + list(self.output.named_parameters()) 
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]

        transformer_parms = [(n,p) for n,p in self.transformer.named_parameters() if any([str(ln) in n for ln in self.layer_nums])] + [(n,p) for n,p in self.named_parameters() if n in ["transformer.encoder.rel_embeddings.weight","transformer.encoder.LayerNorm.weight","transformer.encoder.LayerNorm.bias"]]
        optimizer_parameters += [
            {
                "params": [p for n, p in transformer_parms if not any(nd in n for nd in no_decay)],
                "weight_decay": 0.001,
                "lr" : 2e-5
            },
            {
                "params": [p for n, p in transformer_parms if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr" : 2e-5
            },
        ]
        opt = torch.optim.AdamW(optimizer_parameters, lr=CFG.lr)
        sch = get_polynomial_decay_schedule_with_warmup(
          opt,
          num_warmup_steps=int(num_train_steps * CFG.warmup_ratio),
          num_training_steps=num_train_steps,
          last_epoch=-1,
          )
#         sch = None
        return opt, sch

    def forward(self, ids, mask, token_type_ids=None, targets=None):
        if token_type_ids is not None:
            transformer_out = self.transformer( ids, mask, token_type_ids )
        else:
            transformer_out = self.transformer(ids, mask)
        sequence_output = transformer_out.last_hidden_state[:,0,:]
        # sequence_output = self.meanpooling(transformer_out.last_hidden_state[:,:CFG.n_tokens,:], mask[:,:CFG.n_tokens])
        logits = self.output(sequence_output)
        
        if targets is not None:
            loss = self.loss(logits, targets)
            return loss, logits
        else:
            return None, logits

In [10]:
class Model2(nn.Module):
    def __init__(self, model_name):
        super(Model2, self).__init__()
        
        self.model_name = model_name

        hidden_dropout_prob: float = CFG.hidden_dropout_prob
        layer_norm_eps: float = 1e-7

        config = AutoConfig.from_pretrained(model_name)

        config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": hidden_dropout_prob,
                "attention_probs_dropout_prob" : hidden_dropout_prob,
                "layer_norm_eps": layer_norm_eps,
                "add_pooling_layer": False,
                "num_labels": 6,
            }
        )
        
        self.config = config
        self.transformer = AutoModel.from_pretrained(model_name, config=config)
        self.output = nn.Linear(config.hidden_size, 6)

    def forward(self, ids, mask, token_type_ids=None , targets = None):
        
        if token_type_ids is not None:
            transformer_out = self.transformer( ids, mask, token_type_ids )
        else:
            transformer_out = self.transformer(ids, mask)
            
        sequence_output = transformer_out.last_hidden_state[:,0,:]
        
        logits = self.output(sequence_output)

        return None, logits

In [11]:
model_grps = {
    'deberta_v3_512':{
        'arch': partial( Model, model_name='../input/deberta-v3-large/deberta-v3-large'),
        'tokenizer_path': CFG.tokenizer_deberta_v3_large,
        'checkpoints':[
           "../input/colab-models-download-deberta-large/deberta-v3-large/deberta-v3-large_fold_0.pth",
        "../input/colab-models-download-deberta-large/deberta-v3-large/deberta-v3-large_fold_1.pth",
        "../input/colab-models-download-deberta-large/deberta-v3-large/deberta-v3-large_fold_2.pth",
        "../input/colab-models-download-deberta-large/deberta-v3-large/deberta-v3-large_fold_3.pth",
        "../input/colab-models-download-deberta-large/deberta-v3-large/deberta-v3-large_fold_4.pth"
        ],
        'max_length' : 512,
        'prompt' : False
    },
    
    'deberta_512':{
        'arch': partial( Model, model_name='../input/deberta-large'),
        'tokenizer_path': CFG.tokenizer_deberta_large,
        'checkpoints':[
           '../input/fb3-gpu-train-debertalarge-llrd-rein/deberta-large_fold_0.pth',
            '../input/fb3-gpu-train-debertalarge-llrd-rein/deberta-large_fold_1.pth',
            '../input/fb3-gpu-train-debertalarge-llrd-rein/deberta-large_fold_2.pth',
            '../input/fold3fb3-gpu-train-debertalarge-llrd-rein/deberta-large_fold_3.pth',
            '../input/fold3fb3-gpu-train-debertalarge-llrd-rein/deberta-large_fold_4.pth'
        ],
        'max_length' : 512,
        'prompt' : False
    },
    'electra_v3_512':{
        'arch': partial( Model, model_name='../input/google-electra-large-discriminator'),
        'tokenizer_path': CFG.tokenizer_electra_large,
        'checkpoints':[
           '../input/fb3-gpu-train-electra-llrd-reinit/electra-large-discriminator_fold_0.pth',
            '../input/fb3-gpu-train-electra-llrd-reinit/electra-large-discriminator_fold_1.pth',
            '../input/fb3-gpu-train-electra-llrd-reinit/electra-large-discriminator_fold_2.pth',
            '../input/fb3-gpu-train-electra-llrd-reinit/electra-large-discriminator_fold_3.pth',
            '../input/fb3-gpu-train-electra-llrd-reinit/electra-large-discriminator_fold_4.pth'
        ],
        'max_length' : 512,
        'prompt' : False
    },
    'prompt_deberta_v3_large_768':{
        'arch': partial( PromptModel, model_name='../input/deberta-v3-large/deberta-v3-large'),
        'tokenizer_path':  CFG.tokenizer_deberta_v3_large,
        'checkpoints':[
            '../input/fb3-gputrain-prompttuning-pl-ensb/deberta-v3-large/deberta-v3-large_fold_0.pth',
            '../input/fb3-gputrain-prompttuning-pl-ensb/deberta-v3-large/deberta-v3-large_fold_1.pth',
            '../input/fb3-gputrain-prompttuning-pl-ensb/deberta-v3-large/deberta-v3-large_fold_2.pth',
            '../input/fb3-gputrain-prompttuning-pl-ensb/deberta-v3-large/deberta-v3-large_fold_3.pth',
            '../input/fb3-gputrain-prompttuning-pl-ensb/deberta-v3-large/deberta-v3-large_fold_4.pth',
        ],
        'max_length' : 768 - CFG.n_tokens,
        'prompt' : True
    },
    'prompt_deberta_large_512':{
        'arch': partial( PromptModel, model_name='../input/deberta-large'),
        'tokenizer_path': CFG.tokenizer_deberta_large,
        'checkpoints':[
           '../input/d-large-fb3-gputrain-prompttuning-pl-ensb/deberta-large/deberta-large_fold_0.pth',
            '../input/d-large-fb3-gputrain-prompttuning-pl-ensb/deberta-large/deberta-large_fold_1.pth',
            '../input/d-large-fb3-gputrain-prompttuning-pl-ensb/deberta-large/deberta-large_fold_2.pth',
            '../input/d-large-fb3-gputrain-prompttuning-pl-ensb/deberta-large/deberta-large_fold_3.pth',
            '../input/d-large-fb3-gputrain-prompttuning-pl-ensb/deberta-large/deberta-large_fold_4.pth',
        ],
        'max_length' : 512 - CFG.n_tokens,
        'prompt' : True
    },
    'deberta_large_512':{
        'arch': partial( Model2, model_name='../input/deberta-large'),
        'tokenizer_path': CFG.tokenizer_deberta_large,
        'checkpoints':[
           '../input/derberta-v1-pl-model-train/deberta-large_fold_0.pth',
            '../input/derberta-v1-pl-model-train/deberta-large_fold_1.pth',
            '../input/derberta-v1-pl-model-train/deberta-large_fold_2.pth',
            '../input/derberta-v1-pl-model-train/deberta-large_fold_3.pth',
            '../input/derberta-v1-pl-model-train/deberta-large_fold_4.pth'
        ],
        'max_length' : 512,
        'prompt' : False
    },
     'deberta_large_rmse_512':{
        'arch': partial( Model2, model_name='../input/deberta-large'),
        'tokenizer_path': CFG.tokenizer_deberta_large,
        'checkpoints':[
           '../input/rmse-derberta-v1-pl-model-train/deberta-large_fold_0.pth',
            '../input/rmse-derberta-v1-pl-model-train/deberta-large_fold_1.pth',
            '../input/rmse-derberta-v1-pl-model-train/deberta-large_fold_2.pth',
            '../input/rmse-derberta-v1-pl-model-train/deberta-large_fold_3.pth',
            '../input/rmse-derberta-v1-pl-model-train/deberta-large_fold_4.pth'
        ],
        'max_length' : 512,
        'prompt' : False
    },
    'deberta_v3_large_512':{
         'arch': partial( Model2, model_name='../input/deberta-v3-large/deberta-v3-large'),
        'tokenizer_path':  CFG.tokenizer_deberta_v3_large,
        'checkpoints':[
           '../input/derberta-v3-pl-model-train/deberta-large_fold_0.pth',
            '../input/derberta-v3-pl-model-train/deberta-large_fold_1.pth',
            '../input/derberta-v3-pl-model-train/deberta-large_fold_2.pth',
            '../input/derberta-v3-pl-model-train/deberta-large_fold_3.pth',
            '../input/derberta-v3-pl-model-train/deberta-large_fold_4.pth'
        ],
        'max_length' : 512,
        'prompt' : False
    },
    'deberta_v3_large_rmse_512':{
         'arch': partial( Model2, model_name='../input/deberta-v3-large/deberta-v3-large'),
        'tokenizer_path':  CFG.tokenizer_deberta_v3_large,
        'checkpoints':[
           '../input/rmse-derberta-v3-pl-model-train/deberta-large_fold_0.pth',
            '../input/rmse-derberta-v3-pl-model-train/deberta-large_fold_1.pth',
            '../input/rmse-derberta-v3-pl-model-train/deberta-large_fold_2.pth',
            '../input/rmse-derberta-v3-pl-model-train/deberta-large_fold_3.pth',
            '../input/rmse-derberta-v3-pl-model-train/deberta-large_fold_4.pth'
        ],
        'max_length' : 512,
        'prompt' : False
    }
    
}

In [12]:
test = pd.read_csv("../input/feedback-prize-english-language-learning/test.csv")
submission = pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv")

In [13]:
def _prepare_training_data_helper(tokenizer, df, max_len , prompt):
    training_samples = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        idx = row["text_id"]

        text = row['full_text']
        
        if max_len is not None:
            text = resolve_encodings_and_normalize(text)
            encoded_text = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            return_attention_mask=True,
            truncation="longest_first",
        )
        else:
            encoded_text = tokenizer.encode_plus(
                text,
                None,
                add_special_tokens=True,
                return_attention_mask=True
            )
        input_ids = encoded_text["input_ids"]
        if prompt:
            sample = {
                "text_id": row["text_id"],
                "input_ids": [50256] * CFG.n_tokens + input_ids,
                "attention_mask":[1] * CFG.n_tokens + encoded_text['attention_mask']

            }
        else:
            sample = {
                "text_id": row["text_id"],
                "input_ids": input_ids,
                "attention_mask": encoded_text['attention_mask']

            }


        training_samples.append(sample)
    return training_samples


def prepare_training_data(df, tokenizer, num_jobs, max_len, prompt):
    training_samples = []

    df_splits = np.array_split(df, num_jobs)

    results = Parallel(n_jobs=num_jobs, backend="multiprocessing")(
        delayed(_prepare_training_data_helper)( tokenizer, df, max_len, prompt) for df in df_splits
    )
    for result in results:
        training_samples.extend(result)

    return training_samples


class FeedbackDataset:
    def __init__(self, samples,  tokenizer):
        self.samples = samples
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return {
            "ids": self.samples[idx]["input_ids"],
            "mask": self.samples[idx]["attention_mask"],
        }


class Collate:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        output = dict()
        output["ids"] = [sample["ids"] for sample in batch]
        output["mask"] = [sample["mask"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["ids"]]
            output["mask"] = [s + (batch_max - len(s)) * [0] for s in output["mask"]]
        else:
            output["ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["ids"]]
            output["mask"] = [(batch_max - len(s)) * [0] + s for s in output["mask"]]

        # convert to tensors
        output["ids"] = torch.tensor(output["ids"], dtype=torch.long)
        output["mask"] = torch.tensor(output["mask"], dtype=torch.long)

        return output

In [14]:
def inference_one_epoch(model, dataloader, device):
    model.eval()
    pred = []
    model.to(device)
    model = nn.DataParallel(model)
    for step, data in tqdm( enumerate(dataloader), total=len(dataloader)):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        with torch.no_grad():
            with torch.cuda.amp.autocast(enabled=CFG.fp16):
                _, outputs = model(ids, mask)
                pred.append(np.clip(outputs.to('cpu').numpy(), 1.0, 5.0))
    pred = np.concatenate(pred, axis = 0)
    return pred

In [15]:
device = torch.device('cuda:0')
# device = torch.device('cpu')

In [16]:
weights = [[0.12297735, 0.18924966, 0.12348493, 0.1654967 , 0.03663227,
        0.0052988 ],
       [0.20638024, 0.06784801, 0.13565758, 0.06039658, 0.08407076,
        0.08675586],
       [0.03234911, 0.06139988, 0.17400488, 0.21016095, 0.00592178,
        0.02796648],
       [0.03166416, 0.0842618 , 0.16850583, 0.09228587, 0.28242218,
        0.21844554],
       [0.22140639, 0.06732294, 0.14839885, 0.00893231, 0.06608499,
        0.02334413],
       [0.0904312 , 0.19762827, 0.09182856, 0.04628394, 0.04525437,
        0.24961744],
       [0.08797978, 0.12667792, 0.00323598, 0.068548  , 0.02114802,
        0.02036566],
       [0.19697611, 0.0542199 , 0.13103443, 0.24676037, 0.27021692,
        0.11989797],
       [0.00983565, 0.15139163, 0.02384896, 0.10113529, 0.18824872,
        0.24830812]]
weights = np.array(weights)
# weights = np.ones((9,6))
# weights /= weights.sum(axis = 0)
print(weights)
print(weights.shape)

[[0.12297735 0.18924966 0.12348493 0.1654967  0.03663227 0.0052988 ]
 [0.20638024 0.06784801 0.13565758 0.06039658 0.08407076 0.08675586]
 [0.03234911 0.06139988 0.17400488 0.21016095 0.00592178 0.02796648]
 [0.03166416 0.0842618  0.16850583 0.09228587 0.28242218 0.21844554]
 [0.22140639 0.06732294 0.14839885 0.00893231 0.06608499 0.02334413]
 [0.0904312  0.19762827 0.09182856 0.04628394 0.04525437 0.24961744]
 [0.08797978 0.12667792 0.00323598 0.068548   0.02114802 0.02036566]
 [0.19697611 0.0542199  0.13103443 0.24676037 0.27021692 0.11989797]
 [0.00983565 0.15139163 0.02384896 0.10113529 0.18824872 0.24830812]]
(9, 6)


In [17]:
final_predictions = []

for model_type,model in model_grps.items():
    
    print(f'Model Type : {model_type}')
    
    predictions = []
    
    print('[INFO] Preparing Data...')
    
    tokenizer = model['tokenizer_path']
    samples = prepare_training_data(test, tokenizer, num_jobs=CFG.num_workers, max_len=model['max_length'], prompt = model["prompt"])
    samples = list(sorted(samples, key=lambda d: len(d["input_ids"])))
    dataset = FeedbackDataset(samples, tokenizer)
    collate_fn = Collate(tokenizer)
    
    test_loader = DataLoader(dataset, 
                         batch_size=CFG.batch_size,
                         shuffle=False,
                         collate_fn = collate_fn,
                         num_workers = CFG.num_workers,
                         pin_memory = True,
                         drop_last = False,
                         )
    
    print('[INFO] Inferring....')
    

    for chkpoint in model['checkpoints']:
        net =  model['arch']()

        state = torch.load(chkpoint,
                           map_location = torch.device('cpu'))
        net.load_state_dict(state)
#         net.eval()
#         quantized_model = torch.quantization.quantize_dynamic(net, {torch.nn.Linear}, dtype=torch.qint8)
        prediction = inference_one_epoch(net, test_loader, device)
        predictions.append(prediction)
        del net, state, prediction
        gc.collect()
        torch.cuda.empty_cache()
    
    final_predictions.append( ( [x["text_id"] for x in samples], np.mean(predictions, axis=0) ) )
    
    del dataset, samples, test_loader
    gc.collect()
    
    print('[INFO] Inference Complete.\n\n')
    
    predictions.clear()
    del tokenizer;

Model Type : deberta_v3_512
[INFO] Preparing Data...
[INFO] Inferring....


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Inference Complete.




In [18]:
print( 'Final Predictions Length :',len(final_predictions) )

Final Predictions Length : 9


In [19]:
# weights = np.ones((9,6))
# weights /= weights.sum(axis = 0)

In [20]:
#logits →　probability
sample_submission = pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv")
sample_submission = sample_submission.set_index('text_id')

for i, ( ids, preds ) in enumerate( final_predictions ):
    
    if i == 0:
        for k,col in enumerate(CFG.target_columns):
            sample_submission.loc[ids, col] = weights[i, k] * preds[:, k]
 
    else:
        for k,col in enumerate(CFG.target_columns):
            sample_submission.loc[ids, col] += weights[i, k] * preds[:, k]


sample_submission = sample_submission.reset_index()

In [21]:
# sample_submission[CFG.target_columns] /= len( final_predictions )

In [22]:
sample_submission.to_csv('submission.csv', index=False)

In [23]:
sample_submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.792938,2.663330,3.026169,2.879776,2.644455,2.556221
1,000BAD50D026,2.681290,2.500122,2.718124,2.426224,2.201027,2.680878
2,00367BB2546B,3.601990,3.355713,3.628044,3.594208,3.387848,3.235748


In [24]:
## 2nd Best public scoring ensemble

In [25]:
class CFG1:
    model = "microsoft/deberta-v3-base"
    path = "../input/0911-deberta-v3-base/"
    base = "../input/fb3models/microsoft-deberta-v3-base/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenzier_deberta_v3_base
    gradient_checkpointing=False
    batch_size=24
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.0
    
class CFG2:
    model = "microsoft/deberta-v3-large"
    path = "../input/0911-deberta-v3-large/"
    base = "../input/fb3models/microsoft-deberta-v3-large/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v3_large
    gradient_checkpointing=False
    batch_size=16
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.0  ##
    
class CFG3:
    model = "microsoft/deberta-v2-xlarge"
    path = "../input/0911-deberta-v2-xlarge/"
    base = "../input/fb3models/microsoft-deberta-v2-xlarge/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v2_xlarge
    gradient_checkpointing=False
    batch_size=4
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.0 

class CFG4:
    model = "microsoft/deberta-v3-base"
    path = "../input/0913-deberta-v3-base-fgm/"
    base = "../input/fb3models/microsoft-deberta-v3-base/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenzier_deberta_v3_base
    gradient_checkpointing=False
    batch_size=24
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.0
    
class CFG5:
    model = "microsoft/deberta-v3-large"
    path = "../input/0914-deberta-v3-large-fgm/"
    base = "../input/fb3models/microsoft-deberta-v3-large/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v3_large
    gradient_checkpointing=False
    batch_size=16
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.0 ##
    
class CFG6:
    model = "microsoft/deberta-v2-xlarge"
    path = "../input/0919-deberta-v2-xlarge/"
    base = "../input/fb3models/microsoft-deberta-v2-xlarge/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v2_xlarge
    gradient_checkpointing=False
    batch_size=4
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.0 #
    
class CFG7:
    model = "microsoft/deberta-v2-xlarge-mnli"
    path = "../input/0919-deberta-v2-xlarge-mnli/"
    base = "../input/fb3models/microsoft-deberta-v2-xlarge/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v2_xlarge
    gradient_checkpointing=False
    batch_size=4
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 3.0  ###
    
class CFG8:
    model = "microsoft/deberta-v3-large"
    path = "../input/0925-deberta-v3-large-unscale/"
    base = "../input/fb3models/microsoft-deberta-v3-large/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v3_large
    gradient_checkpointing=False
    batch_size=16
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.2  ##
    
class CFG9:
    model = "microsoft/deberta-v3-large"
    path = "../input/0926-deberta-v3-large-unscale/"
    base = "../input/fb3models/microsoft-deberta-v3-large/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v3_large
    gradient_checkpointing=False
    batch_size=16
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 1.0 #
    
class CFG10:
    model = "microsoft/deberta-v3-large"
    path = "../input/0927-deberta-v3-large-unscale/"
    base = "../input/fb3models/microsoft-deberta-v3-large/"
    config_path = base + "config/config.json"
    tokenizer = CFG.tokenizer_deberta_v3_large
    gradient_checkpointing=False
    batch_size=16
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=10
    trn_fold=list(range(n_fold))
    num_workers=4
    weight = 0.7 # 17
    
CFG_list = [CFG1, CFG2, CFG3, CFG4, CFG5, CFG6, CFG7, CFG8, CFG9, CFG10]

In [26]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores

def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores

def get_logger(filename='inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [27]:
# # ====================================================
# # oof
# # ====================================================
# for CFG in CFG_list:
#     oof_df = pd.read_pickle(CFG.path+'oof_df.pkl')
#     labels = oof_df[CFG.target_cols].values
#     preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
#     score, scores = get_score(labels, preds)
#     LOGGER.info(f'Model: {CFG.model} Score: {score:<.4f}  Scores: {scores}')

In [28]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        #max_length=CFG.max_len,
        #pad_to_max_length=True,
        #truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs

In [29]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class MaxPooling(nn.Module):
    def __init__(self):
        super(MaxPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        embeddings = last_hidden_state.clone()
        embeddings[input_mask_expanded == 0] = -1e4
        max_embeddings, _ = torch.max(embeddings, dim = 1)
        return max_embeddings
    
class MinPooling(nn.Module):
    def __init__(self):
        super(MinPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        embeddings = last_hidden_state.clone()
        embeddings[input_mask_expanded == 0] = 1e-4
        min_embeddings, _ = torch.min(embeddings, dim = 1)
        return min_embeddings
        

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = AutoConfig.from_pretrained(config_path, output_hidden_states=True)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 6)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, input_ids,token_type_ids, attention_mask):
        outputs = self.model(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, attention_mask)
        return feature

    def forward(self, input_ids,token_type_ids, attention_mask):
        feature = self.feature(input_ids,token_type_ids, attention_mask)
        output = self.fc(feature)
        return output

In [30]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    model = nn.DataParallel(model)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            with torch.cuda.amp.autocast(enabled=CFG.fp16):
                y_preds = model(inputs['input_ids'], inputs['token_type_ids'], inputs['attention_mask'])
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [31]:
for _idx, cfg in enumerate(CFG_list):
    test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
    submission = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')
    # sort by length to speed up inference
    test['tokenize_length'] = [len(cfg.tokenizer(text)['input_ids']) for text in test['full_text'].values]
    test = test.sort_values('tokenize_length', ascending=True).reset_index(drop=True)

    test_dataset = TestDataset(cfg, test)
    test_loader = DataLoader(test_dataset,
                             batch_size=cfg.batch_size,
                             shuffle=False,
                             collate_fn=DataCollatorWithPadding(tokenizer=cfg.tokenizer, padding='longest'),
                             num_workers=cfg.num_workers, pin_memory=True, drop_last=False)
    predictions = []
    for fold in cfg.trn_fold:
        model = CustomModel(cfg, config_path=cfg.config_path, pretrained=False)
        state = torch.load(cfg.path+f"{cfg.model.replace('/', '-')}_fold{fold}_best.pth",
                           map_location=torch.device('cpu'))
        model.load_state_dict(state['model'])
        prediction = inference_fn(test_loader, model, device)
        predictions.append(prediction)
        del model, state, prediction; gc.collect()
        torch.cuda.empty_cache()
    predictions = np.mean(predictions, axis=0)
    test[cfg.target_cols] = predictions
    submission = submission.drop(columns=cfg.target_cols).merge(test[['text_id'] + cfg.target_cols], on='text_id', how='left')
    display(submission.head())
    submission[['text_id'] + cfg.target_cols].to_csv(f'submission_{_idx + 1}.csv', index=False)
    del test, submission, predictions, test_dataset, test_loader; gc.collect()
    torch.cuda.empty_cache() 

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiproc

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiproc

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

  0%|          | 0/1 [00:00<?, ?it/s]

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 263, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multipro

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.939453,2.767578,3.113281,2.957031,2.632812,2.601562
1,000BAD50D026,2.681641,2.451172,2.726562,2.408203,2.121094,2.677734
2,00367BB2546B,3.478516,3.294922,3.531250,3.515625,3.337891,3.160156


In [32]:
test = pd.read_csv('../input/feedback-prize-english-language-learning/test.csv')
submission = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')

sub1 = pd.read_csv(f'submission_1.csv')[CFG1.target_cols] * CFG1.weight
sub2 = pd.read_csv(f'submission_2.csv')[CFG2.target_cols] * CFG2.weight
sub3 = pd.read_csv(f'submission_3.csv')[CFG3.target_cols] * CFG3.weight
sub4 = pd.read_csv(f'submission_4.csv')[CFG4.target_cols] * CFG4.weight
sub5 = pd.read_csv(f'submission_5.csv')[CFG5.target_cols] * CFG5.weight
sub6 = pd.read_csv(f'submission_6.csv')[CFG6.target_cols] * CFG6.weight
sub7 = pd.read_csv(f'submission_7.csv')[CFG7.target_cols] * CFG7.weight
sub8 = pd.read_csv(f'submission_8.csv')[CFG8.target_cols] * CFG8.weight
sub9 = pd.read_csv(f'submission_9.csv')[CFG9.target_cols] * CFG9.weight
sub10 = pd.read_csv(f'submission_10.csv')[CFG10.target_cols] * CFG10.weight

ens = ((sub1 + sub2 + sub3 + sub4 + sub5 + sub6 + sub7 + sub8 + sub9 + sub10)
       /(CFG1.weight + CFG2.weight + CFG3.weight + CFG4.weight + CFG5.weight 
         + CFG6.weight + CFG7.weight + CFG8.weight + CFG9.weight + CFG10.weight))

#ens = (sub1 + sub2)/(CFG1.weight + CFG2.weight)

submission[CFG1.target_cols] = ens
display(submission.head())
submission.to_csv('submission.csv', index=False)

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.968824,2.824168,3.129151,2.974782,2.679588,2.667513
1,000BAD50D026,2.702370,2.522546,2.757672,2.430437,2.192832,2.684975
2,00367BB2546B,3.569218,3.357723,3.584782,3.531076,3.368706,3.268924


In [33]:
## Checking that the ordering of rows match with each other
print(submission.text_id == sample_submission.text_id)

0    True
1    True
2    True
Name: text_id, dtype: bool


In [34]:
# final_submission = pd.read_csv('../input/feedback-prize-english-language-learning/sample_submission.csv')
# final_submission[CFG.target_columns] = (submission[CFG.target_columns].values + sample_submission[CFG.target_columns].values)/ 2
# final_submission.to_csv('submission.csv', index=False)

In [35]:
# final_submission.head()

In [36]:
import numpy as np 
import pandas as pd 
import os, gc, re, warnings
warnings.filterwarnings("ignore")

In [37]:
dftr = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/train.csv")
dftr["src"]="train"
dfte = pd.read_csv("/kaggle/input/feedback-prize-english-language-learning/test.csv")
dfte["src"]="test"
print('Train shape:',dftr.shape,'Test shape:',dfte.shape,'Test columns:',dfte.columns)
df = pd.concat([dftr,dfte],ignore_index=True)

dftr.head()

Train shape: (3911, 9) Test shape: (3, 3) Test columns: Index(['text_id', 'full_text', 'src'], dtype='object')


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,src
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,train
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,train
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,train
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,train
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,train


In [38]:
target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions',]

In [39]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
FOLDS = 25
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
for i,(train_index, val_index) in enumerate(skf.split(dftr,dftr[target_cols])):
    dftr.loc[val_index,'FOLD'] = i
print('Train samples per fold:')
dftr.FOLD.value_counts()

Train samples per fold:


11.0    157
21.0    157
1.0     157
7.0     157
20.0    157
14.0    157
19.0    157
12.0    157
23.0    157
3.0     157
6.0     157
5.0     156
13.0    156
22.0    156
0.0     156
15.0    156
24.0    156
17.0    156
9.0     156
8.0     156
4.0     156
2.0     156
18.0    156
10.0    156
16.0    156
Name: FOLD, dtype: int64

In [40]:
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )


BATCH_SIZE = 4

class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.df = df.reset_index(drop=True)
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"full_text"]
        tokens = tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding='max_length',
                truncation=True,
                max_length=MAX_LEN,return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens

ds_tr = EmbedDataset(dftr)
embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)
ds_te = EmbedDataset(dfte)
embed_dataloader_te = torch.utils.data.DataLoader(ds_te,\
                        batch_size=BATCH_SIZE,\
                        shuffle=False)

In [41]:
tokenizer = None
MAX_LEN = 640

def get_embeddings(MODEL_NM='', MAX=640, BATCH_SIZE=4, verbose=True):
    global tokenizer, MAX_LEN
    DEVICE="cuda"
    model = AutoModel.from_pretrained( MODEL_NM )
    tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
    MAX_LEN = MAX
    
    model = model.to(DEVICE)
    model.eval()
    all_train_text_feats = []
    for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)
    if verbose:
        print('Train embeddings shape',all_train_text_feats.shape)
        
    te_text_feats = []
    for batch in tqdm(embed_dataloader_te,total=len(embed_dataloader_te)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        with torch.no_grad():
            model_output = model(input_ids=input_ids,attention_mask=attention_mask)
        sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
        # Normalize the embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
        te_text_feats.extend(sentence_embeddings)
    te_text_feats = np.array(te_text_feats)
    if verbose:
        print('Test embeddings shape',te_text_feats.shape)
        
    return all_train_text_feats, te_text_feats

In [42]:
MODEL_NM = '../input/huggingface-deberta-variants/deberta-base/deberta-base'
all_train_text_feats, te_text_feats = get_embeddings(MODEL_NM)

100%|██████████| 978/978 [04:58<00:00,  3.28it/s]


Train embeddings shape (3911, 768)


100%|██████████| 1/1 [00:00<00:00,  4.18it/s]

Test embeddings shape (3, 768)


In [43]:
MODEL_NM = '../input/deberta-v3-large/deberta-v3-large'
all_train_text_feats2, te_text_feats2 = get_embeddings(MODEL_NM)

100%|██████████| 978/978 [14:03<00:00,  1.16it/s]


Train embeddings shape (3911, 1024)


100%|██████████| 1/1 [00:00<00:00,  1.49it/s]

Test embeddings shape (3, 1024)


In [44]:
MODEL_NM = '../input/huggingface-deberta-variants/deberta-large/deberta-large'
all_train_text_feats3, te_text_feats3 = get_embeddings(MODEL_NM)

100%|██████████| 978/978 [14:29<00:00,  1.13it/s]


Train embeddings shape (3911, 1024)


100%|██████████| 1/1 [00:00<00:00,  1.47it/s]

Test embeddings shape (3, 1024)


In [45]:
MODEL_NM = '../input/huggingface-deberta-variants/deberta-large-mnli/deberta-large-mnli'
all_train_text_feats4, te_text_feats4 = get_embeddings(MODEL_NM, MAX=512)

100%|██████████| 978/978 [11:18<00:00,  1.44it/s]


Train embeddings shape (3911, 1024)


100%|██████████| 1/1 [00:00<00:00,  1.90it/s]

Test embeddings shape (3, 1024)


In [46]:
MODEL_NM = '../input/huggingface-deberta-variants/deberta-xlarge/deberta-xlarge'
all_train_text_feats5, te_text_feats5 = get_embeddings(MODEL_NM, MAX=512)

100%|██████████| 978/978 [21:58<00:00,  1.35s/it]


Train embeddings shape (3911, 1024)


100%|██████████| 1/1 [00:01<00:00,  1.04s/it]

Test embeddings shape (3, 1024)


In [47]:
all_train_text_feats = np.concatenate([all_train_text_feats,all_train_text_feats2,
                                       all_train_text_feats3,all_train_text_feats4,
                                       all_train_text_feats5],axis=1)

te_text_feats = np.concatenate([te_text_feats,te_text_feats2,
                                te_text_feats3,te_text_feats4,
                                te_text_feats5],axis=1)

del all_train_text_feats2, te_text_feats2
del all_train_text_feats3, te_text_feats3
del all_train_text_feats4, te_text_feats4
del all_train_text_feats5, te_text_feats5
gc.collect()

print('Our concatenated embeddings have shape', all_train_text_feats.shape )

Our concatenated embeddings have shape (3911, 4864)


In [48]:
from cuml.svm import SVR
import cuml
print('RAPIDS version',cuml.__version__)


from sklearn.metrics import mean_squared_error

preds = []
scores = []
def comp_score(y_true,y_pred):
    rmse_scores = []
    for i in range(len(target_cols)):
        rmse_scores.append(np.sqrt(mean_squared_error(y_true[:,i],y_pred[:,i])))
    return np.mean(rmse_scores)

#for fold in tqdm(range(FOLDS),total=FOLDS):
for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)
    
    dftr_ = dftr[dftr["FOLD"]!=fold]
    dfev_ = dftr[dftr["FOLD"]==fold]
    
    tr_text_feats = all_train_text_feats[list(dftr_.index),:]
    ev_text_feats = all_train_text_feats[list(dfev_.index),:]
    
    ev_preds = np.zeros((len(ev_text_feats),6))
    test_preds = np.zeros((len(te_text_feats),6))
    for i,t in enumerate(target_cols):
        print(t,', ',end='')
        clf = SVR(C=1)
        clf.fit(tr_text_feats, dftr_[t].values)
        ev_preds[:,i] = clf.predict(ev_text_feats)
        test_preds[:,i] = clf.predict(te_text_feats)
    print()
    score = comp_score(dfev_[target_cols].values,ev_preds)
    scores.append(score)
    print("Fold : {} RSME score: {}".format(fold,score))
    preds.append(test_preds)
    
print('#'*25)
print('Overall CV RSME =',np.mean(scores))

RAPIDS version 21.10.02
#########################
### Fold 1
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 0 RSME score: 0.4594028704643705
#########################
### Fold 2
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 1 RSME score: 0.44688002668894744
#########################
### Fold 3
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 2 RSME score: 0.45990487880948355
#########################
### Fold 4
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 3 RSME score: 0.44741217535307204
#########################
### Fold 5
#########################
cohesion , syntax , vocabulary , phraseology , grammar , conventions , 
Fold : 4 RSME score: 0.45173804796077904
#########################
### Fold 6
#########################
cohesion , syntax , vocabulary , p

In [49]:
sub = dfte.copy()

sub.loc[:,target_cols] = np.average(np.array(preds),axis=0) #,weights=[1/s for s in scores]
sub_columns = pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv").columns
sub = sub[sub_columns]

In [50]:
final_sub = pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv")
final_sub[target_cols] = (submission[target_cols].values + sample_submission[target_cols].values + sub[target_cols].values)/3

In [51]:
final_sub.head(10)

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.893948,2.761994,3.096917,2.931886,2.661745,2.630753
1,000BAD50D026,2.690711,2.488733,2.722679,2.387144,2.135629,2.669383
2,00367BB2546B,3.606480,3.394969,3.599169,3.593889,3.388012,3.287832


In [52]:
final_sub.to_csv('submission.csv', index=False)